関数型のスタイルでプログラミングをする場合、イテレータが重要な基礎となる。  
- イテレータとは
  - 連続データを表現するオブジェクト
  - 一度に一つづつ要素を返す
  - 引数を取らずに次の要素を返すメソッドを必ずサポートしている
    - `__next__()`
      - データストリームに要素が残っていなければStopIteration例外を返す
  - 必要に応じて無限のイテレータを書くこともある



- ビルトインのiter()
  - 任意のオブジェクトを受け取り、その中身や要素を返すイテレータを返す。
  - 渡したオブジェクトがイテレータを作れないものであった場合はTypeErrorを返す
    - リストや辞書などのイテラブルなオブジェクト

In [1]:
L = [1,2,3]
it = iter(L)
it

In [2]:
for i in it:
    print(i)

1
2
3


前のセルでオブジェクトの中身を使い果たしているのでStopIteration

In [3]:
next(it)

StopIteration: 

イテレータはコンストラクタ関数でリストやタプルに具現化できる


In [4]:
iterator = iter(L)
t = tuple(iterator)
t

(1, 2, 3)

In [5]:
# アンパック
a,b,c, = t
a,b,c

(1, 2, 3)

シーケンス型はどれでも自動でイテレータ生成に対応している

In [6]:
# 3.7から辞書の反復順序は挿入順序と同じで有ることが保証された
m = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12,
}
for key in m:
    print(key, m[key])

Jan 1
Feb 2
Mar 3
Apr 4
May 5
Jun 6
Jul 7
Aug 8
Sep 9
Oct 10
Nov 11
Dec 12


辞書はキーでイテレータを回すが、明示的に値やキーあるいはそれらのペアでイテレートしたい場合は`values()`や`items()`メソッドでイテレータを作ることができる。


`dict()`は`(key,value)`タプルのストリームを返すイテレータを受け入れることができる。

In [7]:
l = [("Italy", "Rome"), ("France", "Paris"), ("US", "Washington DC")]
dict(iter(l))

{'Italy': 'Rome', 'France': 'Paris', 'US': 'Washington DC'}

セットもイテラブルを受け取れるし、要素でイテレートできる。

In [8]:

S = {2,3,5,7,11,13}
for i in S:
    print(i)

2
3
5
7
11
13


## ジェネレータ式とリスト内包表記
- イテレータによく使う操作
  - 一つづつ全要素に操作を実行する
  - 条件の合う要素でサブセットを作る



下記は文字列のストリームから空白を取り除くコード

In [9]:
line_list = ["   line 1\n", "line 2   \n", "  \n", ""]


In [10]:
from typing import Generator


striped_iter: Generator[str, None, None] = (line.strip() for line in line_list)

In [11]:
striped_list: list[str] = [line.strip() for line in line_list]

In [12]:
# if条件式で特定の要素を抜き取る
striped_list2: list[str] = [line.strip() for line in line_list if line == ""]

In [13]:
seq1 = "abc"
seq2 =(1,2,3)
seq_marge = [(x,y) for x in seq1 for y in seq2]
seq_marge[0]

('a', 1)

## ジェネレータ
- イテレータを書く作業を簡単にする特殊な関数
  - ジェネレータは一連の値を返す



In [14]:
def generate_ints(N):
    for i in range(N):
        yield i


In [15]:
gen = generate_ints(3)
gen

<generator object generate_ints at 0x10b35d3c0>

In [16]:
next(gen)

0

In [17]:
next(gen)

1

In [18]:
next(gen)

2

In [19]:
next(gen)

StopIteration: 

### ジェネレータに値を渡す
- ジェネレータに値を送るには`send(value)`を使う。
  - ジェネレータのコードが実行を再開し、`yield`が値を返す。

In [21]:
def counter(maximum):
    i = 0
    while i < maximum:
        val = yield i
        if val is not None:
            i = val
        else:
            i += 1


In [23]:
it = counter(10)
next(it)

0

In [24]:
next(it)

1

In [25]:
it.send(8)

8

In [26]:
next(it)

9

In [27]:
next(it)

StopIteration: 

## イテレータと一緒に使われることが多い組み込み関数

In [41]:
# map
def upper(s: str) -> str:
    return s.upper()

list(map(upper, ["sentence", "fragment"]))

['SENTENCE', 'FRAGMENT']

mapに追加のイテラブルを渡す場合、mapにわたした関数の引数とイテラブルの数が同じでなければならない。  
複数のイテラブルが渡された場合、最も短いイテラブルの要素が使い果たされた段階でストップする。

In [47]:
def upper2(a: str,b: str) -> list[str]:
    return [a.upper(), b.upper()]

list(map(upper2, ["sentence", "fragment"], ["hello", "python", "function"]))

[['SENTENCE', 'HELLO'], ['FRAGMENT', 'PYTHON']]

In [48]:
# リスト内包表記でも
[upper(s) for s in ["sentence", "fragment"]]

['SENTENCE', 'FRAGMENT']

In [49]:
[
    upper2(x, y)
    for x in ["sentence", "fragment"]
    for y in ["hello", "python", "function"]
]


[['SENTENCE', 'HELLO'],
 ['SENTENCE', 'PYTHON'],
 ['SENTENCE', 'FUNCTION'],
 ['FRAGMENT', 'HELLO'],
 ['FRAGMENT', 'PYTHON'],
 ['FRAGMENT', 'FUNCTION']]

In [50]:
# フィルタ関数
def is_even(x: int) -> bool:
    return x % 2 == 0

list(filter(is_even, range(10)))

[0, 2, 4, 6, 8]

In [51]:
[x for x in range(10) if x % 2 == 0]

[0, 2, 4, 6, 8]

`enumerate(iter, start=0)`はイテラブルに0ベースのキーを付与して二要素のタプルを返す

In [54]:
for item in enumerate(["a", "b","c"]):
    print(item)

(0, 'a')
(1, 'b')
(2, 'c')


In [56]:
with open("data.txt", "r") as f:
    for i, line in enumerate(f,start=1):
        if line.strip() == "":
            print("Blank line at line #%i" % i)

Blank line at line #3
Blank line at line #7


`sorted(iterable, key=None reverse=False)`は要素をすべて集めたリストを作り、ソートして返す。

In [63]:
import random
rand_list = random.sample(range(10000), 8)
rand_list

[5712, 6292, 253, 8879, 1705, 6035, 8231, 9189]

In [64]:
sorted(rand_list)

[253, 1705, 5712, 6035, 6292, 8231, 8879, 9189]

In [65]:
sorted(rand_list, reverse=True)

[9189, 8879, 8231, 6292, 6035, 5712, 1705, 253]

In [68]:
# any
any(["a","",""])

True

In [71]:
# all
all([1,1,1])

True

In [77]:
# zip
list(zip([1,2,3], ["a", "b", "c", "d"], strict=False))

[(1, 'a'), (2, 'b'), (3, 'c')]

In [76]:
# zip
list(zip([1,2,3], ["a", "b", "c", "d"], strict=True))

ValueError: zip() argument 2 is longer than argument 1